In [1]:

import math
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts
import pandas as pd 
from LinApp_Deriv import LinApp_Deriv
from LinApp_FindSS import LinApp_FindSS
from LinApp_Solve import LinApp_Solve
from rouwen import rouwen
from numba import jit
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Parameters
alpha = 0.35
beta = 0.98
rho = 0.95
sigma = 0.02
num = 26
sizek = num
sizez = num

In [3]:
# Create discrete grid for k
A = alpha*beta
kbar = A**(1/(1-alpha))
kmax = 1.5*kbar
kmin = 0.5*kbar
kgrid = np.linspace(kmin,kmax,sizek)

# Create grid for z 
zmax = 5*sigma
zmin = -5*sigma
mu = (zmax+zmin)/2
step = (zmax-zmin)/(sizez-1)
pi, zgrid = rouwen(rho,mu,step,sizez)

In [6]:
def Modeldyn_BM(theta0, params):
    '''
    Inputs are:
        theta: a vector 
        params: list of parameter values
    
    Output are:
        Euler: a vector of Euler equations written so that they are zero at the
            steady state values of X, Y & Z.  This is a 2x1 numpy array. 
    '''
    
    # unpack theta0
    (Kpp, Kp, K, Zp, Z) = theta0
    
    # unpacking parameters
    alpha, beta = params
    
    # Evaluate Euler equations
    E1 = (alpha*beta*np.exp(Zp)*(np.exp(Kp)**(alpha-1))*
          (np.exp(Z)*(np.exp(K)**alpha)-np.exp(Kp))*
          (1/(np.exp(Zp)*(np.exp(Kp)**alpha)-np.exp(Kpp))))
    
    return np.array([E1])


In [7]:
nx = 1
ny = 0
nz = 1
logX = 1
Sylv = 1

# set up steady state input vector
theta0 = np.array([kbar, kbar, kbar, 0., 0.])
params = (alpha, beta)
Zbar = np.array([0])

# find the derivatives matrices
[AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, WW, TT] = \
    LinApp_Deriv(Modeldyn_BM, params, theta0, nx, ny, nz, logX)
np.set_printoptions(suppress=False)
np.set_printoptions(precision=6)

print('FF: ', FF)
print('GG: ', GG)
print('HH: ', HH)
print('LL: ', LL)
print('MM: ', MM)

# set value for NN    
NN = rho
    
# find the policy and jump function coefficients
PP, QQ, UU, RR, SS, VV = \
    LinApp_Solve(AA,BB,CC,DD,FF,GG,HH,JJ,KK,LL,MM,WW,TT,NN,Zbar,Sylv)
print ('PP:',PP)
print ('QQ', QQ)

FF:  [[-0.380254]]
GG:  [[ 0.468559]]
HH:  [[-0.117414]]
LL:  [[ 2.569321]]
MM:  [[-2.266718]]
Using Sylvester equation solver...
PP: [[ 0.35]]
QQ [[ 6.756849]]
